# Нейронные сети
__Суммарное количество баллов: 10__

В этом домашнем задании можно использовать готовые классы библиотек PyTorch, Keras, TensorFlow.

### Задание 1 (4 балла)

Создайте сеть из трех сверточных слоев по 8 сверток 3х3 и двух полносвязных слоев по 64 нейрона. 

Обучите сеть на датасете mnist с тремя разными функциями активации в слоях (sigmoid, tanh, ReLU).

In [30]:
import pathlib
import string
import keras
import pandas as pd
import numpy as np
import torch

from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [31]:
def read_mnist_dataset(path_to_csv):
    # Возвращает пару из X и y. X - массив векторов. y - соответствующие векторам метки
    df = shuffle(pd.read_csv(path_to_csv))
    y = df['label'].values
    X = df.drop('label', axis=1).values
    return X, y

def train_test_split(X, y, ratio=0.8):
    n = int(len(y) * ratio)
    return X[:n], y[:n], X[n:], y[n:]

In [32]:
num_classes = 10
img_rows, img_cols = 28, 28

X, y = read_mnist_dataset('hw1/mnist.csv')

X_train, y_train, X_test, y_test = train_test_split(X, y)
print(X_train.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


(8000, 784)


In [33]:
def activation_mode(activation):
    batch_size = 256
    epochs = 5

    model = Sequential()
    model.add(Conv2D(8, kernel_size=(3, 3), activation=activation, input_shape=input_shape))
    model.add(Conv2D(8, (3, 3), activation=activation))
    model.add(Conv2D(8, (3, 3), activation=activation))
    model.add(Flatten())
    model.add(Dense(64, activation=activation))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test))
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

### ReLU

In [34]:
activation_mode('relu')

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 3s 400us/step - loss: 1.1853 - accuracy: 0.6219 - val_loss: 0.5106 - val_accuracy: 0.8420
Epoch 2/5
8000/8000 [==============================] - 2s 247us/step - loss: 0.3297 - accuracy: 0.9024 - val_loss: 0.4021 - val_accuracy: 0.8620
Epoch 3/5
8000/8000 [==============================] - 2s 253us/step - loss: 0.1998 - accuracy: 0.9404 - val_loss: 0.2721 - val_accuracy: 0.9160
Epoch 4/5
8000/8000 [==============================] - 2s 258us/step - loss: 0.1393 - accuracy: 0.9575 - val_loss: 0.1647 - val_accuracy: 0.9475
Epoch 5/5
8000/8000 [==============================] - 2s 240us/step - loss: 0.0979 - accuracy: 0.9712 - val_loss: 0.1995 - val_accuracy: 0.9365
Test loss: 0.19950240260362626
Test accuracy: 0.9365000128746033


### Sigmoid

In [35]:
activation_mode('sigmoid')

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 5s 573us/step - loss: 2.3320 - accuracy: 0.1075 - val_loss: 2.3203 - val_accuracy: 0.1000
Epoch 2/5
8000/8000 [==============================] - 4s 481us/step - loss: 2.3053 - accuracy: 0.1023 - val_loss: 2.3064 - val_accuracy: 0.1045
Epoch 3/5
8000/8000 [==============================] - 4s 476us/step - loss: 2.3036 - accuracy: 0.1090 - val_loss: 2.3032 - val_accuracy: 0.1130
Epoch 4/5
8000/8000 [==============================] - 4s 448us/step - loss: 2.3029 - accuracy: 0.1088 - val_loss: 2.3014 - val_accuracy: 0.1130
Epoch 5/5
8000/8000 [==============================] - 4s 452us/step - loss: 2.3022 - accuracy: 0.1130 - val_loss: 2.3054 - val_accuracy: 0.1035
Test loss: 2.305407880783081
Test accuracy: 0.10350000113248825


### Tanh

In [36]:
activation_mode('tanh')

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 5s 587us/step - loss: 0.7468 - accuracy: 0.7900 - val_loss: 0.3755 - val_accuracy: 0.8895
Epoch 2/5
8000/8000 [==============================] - 4s 485us/step - loss: 0.3116 - accuracy: 0.9118 - val_loss: 0.2683 - val_accuracy: 0.9225
Epoch 3/5
8000/8000 [==============================] - 4s 455us/step - loss: 0.2360 - accuracy: 0.9325 - val_loss: 0.2569 - val_accuracy: 0.9305
Epoch 4/5
8000/8000 [==============================] - 4s 460us/step - loss: 0.1900 - accuracy: 0.9449 - val_loss: 0.1988 - val_accuracy: 0.9455
Epoch 5/5
8000/8000 [==============================] - 4s 459us/step - loss: 0.1611 - accuracy: 0.9555 - val_loss: 0.2092 - val_accuracy: 0.9350
Test loss: 0.20918868008255959
Test accuracy: 0.9350000023841858


### Задание 2 (6 баллов)

Достигните точности 87% на test датасете notMNIST.

Архитектура сети может быть любая. Можно использовать Data Augmentation.

In [37]:
def read_not_mnist(letter2label):
    image_arrays = []
    labels = []
    
    dataset_path = pathlib.Path('hw1/notMNIST_small')
    for letter, label in letter2label.items():
        for image_path in (dataset_path / letter).glob('**/*.png'):
            img = load_img(str(image_path))
            x = img_to_array(img)
            image_arrays.append(x)
            labels.append(label)
    X = np.array(image_arrays)
    y = np.array(labels)
    # shuffle
    n = y.size
    idx = np.random.choice(n, n, replace=False)
    return X[idx], y[idx]

In [38]:
batch_size = 128
num_classes = 10
epochs = 10
img_rows, img_cols = 28, 28

letters = list(string.ascii_uppercase[:10])
letter2label = dict(zip(letters, range(10)))
label2letter = dict(zip(range(10), letters))

X, y = read_not_mnist(letter2label)
X_train, y_train, X_test, y_test = train_test_split(X, y)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
input_shape = X_train.shape[1:]
print(input_shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

(28, 28, 3)


In [39]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 25s 2ms/step - loss: 0.7393 - accuracy: 0.7740 - val_loss: 0.3106 - val_accuracy: 0.9031
Epoch 2/10
14979/14979 [==============================] - 24s 2ms/step - loss: 0.3456 - accuracy: 0.9014 - val_loss: 0.2663 - val_accuracy: 0.9178
Epoch 3/10
14979/14979 [==============================] - 25s 2ms/step - loss: 0.2794 - accuracy: 0.9158 - val_loss: 0.2371 - val_accuracy: 0.9303
Epoch 4/10
14979/14979 [==============================] - 24s 2ms/step - loss: 0.2325 - accuracy: 0.9300 - val_loss: 0.2335 - val_accuracy: 0.9327
Epoch 5/10
14979/14979 [==============================] - 25s 2ms/step - loss: 0.2119 - accuracy: 0.9358 - val_loss: 0.2344 - val_accuracy: 0.9300
Epoch 6/10
14979/14979 [==============================] - 24s 2ms/step - loss: 0.1882 - accuracy: 0.9412 - val_loss: 0.2193 - val_accuracy: 0.9402
Epoch 7/10
14979/14979 [==============================] - 24s 2ms/ste